In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.datasets import cifar10

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Dataset

In [2]:
train_xs = np.load('./train_caches.npy')
test_xs = np.load('./test_caches.npy')
(x_train, train_ys), (x_test, test_ys) = cifar10.load_data()
n_classes = 10 
# reshape
test_ys = test_ys.reshape(-1)
train_ys = train_ys.reshape(-1)

# Data provider

In [3]:
import numpy as np
import random

class DataProvider(object):
    def __init__(self, images, labels):
        self.n_sample = len(labels)
        self.queue = list(range(self.n_sample))
        random.shuffle(self.queue)

        self.images = images
        self.labels = labels
        self.epoch_count = 0

    def next_batch(self, batch_size):
        if len(self.queue) < batch_size:
            self.queue = list(range(self.n_sample))
            self.epoch_count += 1
        target_indices = self.queue[:batch_size]
        del self.queue[:batch_size]
        return self.images[target_indices], self.labels[target_indices]

# Modeling

In [4]:
graph = tf.Graph()

with graph.as_default() :
    xs = tf.placeholder(dtype = tf.float32, shape = [None,2048])
    ys = tf.placeholder(dtype = tf.int32 , shape=[None])
    lr = tf.placeholder(dtype = tf.float32, shape = [])
    
    n_units=[1024, 1024]
    layer = xs 
    for ind, units in enumerate(n_units):
        with tf.variable_scope('layer_{}'.format(ind)):
            layer = tf.layers.Dense(units, tf.nn.relu,
                                tf.initializers.he_normal)(layer)
    logits = tf.layers.Dense(n_classes)(layer)

Instructions for updating:
Colocations handled automatically by placer.


# Loss

In [5]:
with graph.as_default() :
    loss = tf.losses.sparse_softmax_cross_entropy(ys, logits)
    loss = tf.identity(loss, 'loss')

Instructions for updating:
Use tf.cast instead.


# Prediction

In [6]:
with graph.as_default() :
    pred = tf.nn.softmax(logits)

## Metric

In [7]:
with graph.as_default() :
    # metric
    logits_cls = tf.argmax(logits, axis=1)
    logits_cls = tf.cast(logits_cls, dtype=tf.int32)
    acc = tf.reduce_mean(tf.cast(tf.equal(ys, logits_cls),tf.float32),
                         name='acc')

## Optimizer

In [8]:
with graph.as_default() :    
    train_op = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(loss)

## Open Session

In [9]:
with graph.as_default() :    

    sess = tf.Session()
    init = tf.group(tf.global_variables_initializer(),
                    tf.local_variables_initializer())
    # saver 
    sess.run(init)

## Training

In [10]:
# hparam 
batch_size = 100
min_loss = 1000000.0
learning_rate = 0.001
max_iter = 10000

train_loss_ch = []
train_acc_ch = []
val_loss_ch = []
val_acc_ch = []

dataprovider = DataProvider(images=train_xs, labels=train_ys)

In [11]:
with graph.as_default() :
    for i in range(max_iter):
        # extract batch_xs, batch_ys
        batch_xs, batch_ys = dataprovider.next_batch(batch_size)

        # Training 
        sess.run(train_op, {xs: batch_xs, ys: batch_ys, lr: learning_rate})
        
        # Validation 
        if i % 100 == 0 :
            # Validate validation dataset 
            fetches=[loss, acc]
            val_loss, val_acc = sess.run(fetches, 
                                         {xs: test_xs, ys: test_ys})
            val_loss_ch.append(val_loss)
            val_acc_ch.append(val_acc)

            # Validate train dataset : extract randomly 10000 samples from train dataset 
            batch_xs, batch_ys = dataprovider.next_batch(10000)
            train_loss, train_acc = sess.run(fetches,{xs: batch_xs, ys: batch_ys})
            train_loss_ch.append(train_loss)
            train_acc_ch.append(train_acc)

            print('step : {} train loss : {:.4f} acc : {:.4f} | Val loss : {:.4f} acc : {:.4f}'.\
            format(i, train_loss, train_acc, val_loss, val_acc))

step : 0 train loss : 2.4125 acc : 0.0856 | Val loss : 2.4113 acc : 0.0844
step : 100 train loss : 0.9789 acc : 0.8103 | Val loss : 0.9786 acc : 0.8085
step : 200 train loss : 0.5891 acc : 0.8379 | Val loss : 0.5929 acc : 0.8376
step : 300 train loss : 0.4803 acc : 0.8549 | Val loss : 0.4941 acc : 0.8483
step : 400 train loss : 0.4483 acc : 0.8547 | Val loss : 0.4512 acc : 0.8536
step : 500 train loss : 0.4102 acc : 0.8699 | Val loss : 0.4257 acc : 0.8612
step : 600 train loss : 0.3959 acc : 0.8702 | Val loss : 0.4106 acc : 0.8636
step : 700 train loss : 0.3851 acc : 0.8724 | Val loss : 0.3982 acc : 0.8666
step : 800 train loss : 0.3622 acc : 0.8830 | Val loss : 0.3908 acc : 0.8697
step : 900 train loss : 0.3676 acc : 0.8751 | Val loss : 0.3815 acc : 0.8705
step : 1000 train loss : 0.3540 acc : 0.8842 | Val loss : 0.3770 acc : 0.8735
step : 1100 train loss : 0.3482 acc : 0.8831 | Val loss : 0.3716 acc : 0.8748
step : 1200 train loss : 0.3461 acc : 0.8853 | Val loss : 0.3657 acc : 0.875

KeyboardInterrupt: 